In [1]:
from pyqode.utils import psi_to_pa, r_to_k, radius_to_area
from frog.doe import DoEGenerator, DoERunner
from skopt.sampler import Lhs
from skopt.space import Space
import numpy as np
from backnozzle.cad import BackNozzleCAD
from pyqode.solver import SU2Solver, Solver
import pandas as pd

/home/ppiper/micromamba/envs/frog/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-10 23:56:52,326	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
def run_2D(config):
    nozzle = BackNozzleCAD(
        divergence_angle=config['divergence_angle'],
        throat_radius=config['throat_radius'],
    )

    points = nozzle.get_points(config['domain_Nx'])
    xn, rn = points[:, 0], points[:, 1]

    config['domain_x'] = xn
    config['domain_area'] = radius_to_area(rn)

    su2 = SU2Solver(config=config)
    su2.run()

    return su2.check_convergence()

def run_1D(config):
    nozzle = BackNozzleCAD(
        divergence_angle=config['divergence_angle'],
        throat_radius=config['throat_radius'],
    )

    points = nozzle.get_points(config['domain_size'])
    xn, rn = points[:, 0], points[:, 1]

    config['domain_x'] = xn
    config['domain_area'] = radius_to_area(rn),

    q1d = Solver(config = config)
    q1d.run()

    return q1d.check_convergence()

In [3]:
variables = {
    'bc_p0' : (psi_to_pa(45.1), psi_to_pa(250.2)),          #P0IN
    'bc_T0' : (r_to_k(530.0), r_to_k(1500.0)),              #T0IN
    'divergence_angle' : (13.0, 17.0),                                 #DIVERGENCE ANGLE
    'bc_Tw' : (0.4*r_to_k(530), 0.65*r_to_k(1500.0))        #WALL TEMPERATURE]
}

other_variables_1D = {
    'working_dir': './data/1D',
    'domain_size': 100,
    'bc_M': 0.0001,
    'bc_pb': 101325,
    'fluid_R': 287.0,
    'fluid_gamma': 1.35,
    'solver_itmax': 50000,
    'solver_itprint': 1,
    'solver_CFL': 0.1,
    'solver_tol': 1e-8,
    'solver_tscheme': 'Euler',
    'solver_fscheme': 'AUSM',
    'solver_dttype': 'Global',
    'solver_dim': 'Dimensionless',  
    'throat_radius': 20.32,
}

other_variables_2D = {
    'working_dir': './data/2D',
    'su2_template': './data/template_su2.cfg',
    'su2_config': 'fluid.cfg',
    'domain_axisymmetric': 'YES',
    'domain_Nx': 100+1,
    'domain_Ny': 18+1,
    'domain_inflation_rate': 1.00,
    'domain_bl_beta': 1.001,
    'domain_bl_nblayers': 7,
    'domain_hwall_n': 0.01,
    'domain_mesh' : f'./mesh/nozzle.su2',
    'turbulence_model': 'SST',
    'solver_dim': 'FREESTREAM_PRESS_EQ_ONE',
    'fluid': 'STANDARD_AIR',
    'fluid_gamma': 1.35,
    'fluid_R': 287.0,
    'fluid_critical_temperature': 131.00,
    'fluid_critical_pressure': 3588550.0,
    'fluid_accentric_factor': 0.035,
    'fluid_viscosity_model': 'SUTHERLAND', 
    'fluid_sutherland_viscosity': 1.716E-5,
    'fluid_sutherland_temperature': 273.15,
    'fluid_sutherland_constant': 110.4,
    'fluid_conductivity_model': 'CONSTANT_PRANDTL',
    'fluid_laminar_prandtl': 0.72,
    'fluid_turbulent_prandtl': 0.9,
    'solver_CFL': 0.01,
    'bc_M': 0.0001,
    'bc_pb': 101325,
    'solver_itmax': 100,
    'solver_tol': np.log10(1e-8),
    'solver_tscheme': 'EULER_IMPLICIT',
    'solver_fscheme': 'JST_KE',
    'solver_dttype': 'Global',
    'output_files': 'PARAVIEW',
    'throat_radius': 20.32  
}

In [4]:
doe_generator = DoEGenerator(
    variables = variables,
    sampler = Lhs(lhs_type="classic", criterion="maximin", iterations=10000),
    n_samples = 1,
)

doe_generator.save('./data/doe_lhs.csv')

In [5]:
doe_generator.df

,bc_p0,bc_T0,divergence_angle,bc_Tw
design_point,,,,
0,1.021992e+06,763.133534,16.533607,241.720275


In [6]:
doe_runner_1D = DoERunner(
    file='./data/doe_lhs.csv', 
    other_variables = other_variables_1D,
    function = run_1D,
    log_file = './data/doe_lhs_1D_log.csv'
)

doe_runner_2D = DoERunner(
    file='./data/doe_lhs.csv', 
    other_variables = other_variables_2D,
    function = run_2D,
    log_file = './data/doe_lhs_2D_log.csv'
)

In [7]:
doe_runner_1D.run()

################################################################################
                           -*- Q1D Euler Solver -*-
Eigen 3.3.7
Allan Moreira de Carvalho
################################################################################
                           -*- Setup Information -*-
################################################################################
# Domain x-coordinates at cell faces (no need for ghost cells)
/home/ppiper/Dropbox/xps/doctorate/repositories/frog/examples/data/1D/0/inputs/xn.txt
# Area distributuin at cell faces (no need for ghost cells)
/home/ppiper/Dropbox/xps/doctorate/repositories/frog/examples/data/1D/0/inputs/sn.txt
# Inlet Total Pressure [Pa]
1021991.6021095794
# Inlet Total Temperature [K]
763.1335340205264
# Inlet Mach Number
0.0001
# Outlet Static Pressure [Pa]
101325
# Gas constant [J/kg/K]
287.0
# Specific heat ratio
1.35
# Maximum number of iterations 
50000
# Interval to print iterations 
1
# CFL number 
0.1
# Convergence

In [8]:
doe_runner_1D.save('./data/doe_lhs_1D_log.csv')